In [1]:
from __future__ import annotations

import math
from collections import Counter, defaultdict
from dataclasses import dataclass, asdict
from pathlib import Path
from typing import Dict, List, Optional

import numpy as np
import pandas as pd


# -----------------------------
# Data structure for one column
# -----------------------------
@dataclass
class ColumnProfile:
    table: str
    column: str
    dtype: str

    n_rows: int
    n_non_null: int

    # Identifier metrics
    completeness: float          # γ(c) = non-null / total
    distinct_count: int          # number of distinct non-null values
    distinctness: float          # u(c) = distinct / non-null
    avg_length: Optional[float]  # average string length (if applicable)
    std_length: Optional[float]  # std dev of string length
    length_stability: float      # λ(c) in [0, 1]
    s_pk: float                  # u * γ * λ

    # Datetime metrics
    parse_success: float         # p(c) = parsed / non-null
    n_parsed: int
    distinct_times: int
    time_diversity: float        # δ(c) = distinct_times / parsed
    time_sd_days: Optional[float]
    time_iqr_days: Optional[float]
    time_entropy: Optional[float]
    s_dt: float                  # weighted percentile score in [0, 1]


# -----------------------------
# Helper functions
# -----------------------------
def entropy_from_counts(counts) -> Optional[float]:
    """Shannon entropy (base 2) from a collection of counts."""
    total = sum(counts)
    if total == 0:
        return None

    ent = 0.0
    for c in counts:
        if c == 0:
            continue
        p = c / total
        ent -= p * math.log2(p)
    return ent


def percentile_ranks(values: List[Optional[float]]) -> List[float]:
    """
    Compute simple percentile ranks in [0,1] for a list of values.

    None/NaN -> 0.0.
    """
    vals = [v for v in values if v is not None and not (isinstance(v, float) and math.isnan(v))]
    if not vals:
        return [0.0 for _ in values]

    sorted_vals = sorted(vals)
    n = len(sorted_vals)

    def rank(v: float) -> float:
        # proportion of values <= v (mid-rank style)
        count = 0
        for sv in sorted_vals:
            if sv <= v:
                count += 1
            else:
                break
        if n == 1:
            return 1.0
        return (count - 1) / (n - 1)

    ranks = []
    for v in values:
        if v is None or (isinstance(v, float) and math.isnan(v)):
            ranks.append(0.0)
        else:
            ranks.append(rank(v))
    return ranks


# -----------------------------
# Column + table profiling
# -----------------------------
def profile_column(table_name: str, col_name: str, series: pd.Series) -> ColumnProfile:
    """Compute all metrics for a single column."""
    n_rows = len(series)
    n_non_null = series.notna().sum()
    completeness = n_non_null / n_rows if n_rows > 0 else 0.0

    # Distinctness u(c)
    if n_non_null > 0:
        distinct_values = series.dropna()
        distinct_count = distinct_values.nunique()
        distinctness = distinct_count / n_non_null
    else:
        distinct_count = 0
        distinctness = 0.0

    # String length stability λ(c)
    avg_len = std_len = None
    if series.dtype == "object" or pd.api.types.is_string_dtype(series):
        lengths = series.dropna().astype(str).str.len()
        if len(lengths) > 0:
            avg_len = float(lengths.mean())
            std_len = float(lengths.std(ddof=0))
            if avg_len > 0:
                cv = std_len / avg_len  # coefficient of variation
                length_stability = 1.0 / (1.0 + cv)
            else:
                length_stability = 1.0
        else:
            length_stability = 0.0
    else:
        # numeric/bool columns usually have fixed-width representations
        length_stability = 1.0

    s_pk = distinctness * completeness * length_stability

    # Datetime parsing
    parsed = pd.to_datetime(series.dropna(), errors="coerce", infer_datetime_format=True)
    n_parsed = parsed.notna().sum()
    parse_success = n_parsed / n_non_null if n_non_null > 0 else 0.0

    distinct_times = parsed.nunique()
    time_diversity = distinct_times / n_parsed if n_parsed > 0 else 0.0

    time_sd_days = time_iqr_days = time_entropy = None
    if n_parsed > 1:
        seconds = parsed.dropna().astype("int64") // 10**9
        seconds = seconds.to_numpy()

        sec_sd = float(seconds.std())
        time_sd_days = sec_sd / (24 * 3600)

        q1 = float(np.percentile(seconds, 25))
        q3 = float(np.percentile(seconds, 75))
        time_iqr_days = (q3 - q1) / (24 * 3600)

        days = (seconds // (24 * 3600)).astype(int)
        counts = Counter(days)
        time_entropy = entropy_from_counts(counts.values())

    return ColumnProfile(
        table=table_name,
        column=col_name,
        dtype=str(series.dtype),
        n_rows=n_rows,
        n_non_null=n_non_null,
        completeness=completeness,
        distinct_count=distinct_count,
        distinctness=distinctness,
        avg_length=avg_len,
        std_length=std_len,
        length_stability=length_stability,
        s_pk=s_pk,
        parse_success=parse_success,
        n_parsed=n_parsed,
        distinct_times=distinct_times,
        time_diversity=time_diversity,
        time_sd_days=time_sd_days,
        time_iqr_days=time_iqr_days,
        time_entropy=time_entropy,
        s_dt=0.0,
    )


def profile_table(table_name: str, df: pd.DataFrame) -> List[ColumnProfile]:
    profiles: List[ColumnProfile] = []
    for col in df.columns:
        profiles.append(profile_column(table_name, col, df[col]))
    return profiles


In [4]:
def assign_datetime_scores(
    profiles: List[ColumnProfile],
    weights: Dict[str, float] | None = None,
    parse_threshold: float = 0.5,
    min_distinct_times: int = 2,
) -> None:
    """
    Compute s_dt(c) for datetime-like columns in each table.
    """
    if weights is None:
        # Default weights – adjust these if you learn them later
        weights = dict(
            completeness=0.25,
            diversity=0.25,
            sd=0.20,
            iqr=0.15,
            entropy=0.15,
        )

    tables: Dict[str, List[ColumnProfile]] = defaultdict(list)
    for p in profiles:
        tables[p.table].append(p)

    for table_name, plist in tables.items():
        candidates = [
            p for p in plist
            if p.parse_success >= parse_threshold
            and p.distinct_times >= min_distinct_times
        ]
        if not candidates:
            continue

        completeness_vals = [p.completeness for p in candidates]
        diversity_vals = [p.time_diversity for p in candidates]
        sd_vals = [p.time_sd_days for p in candidates]
        iqr_vals = [p.time_iqr_days for p in candidates]
        entropy_vals = [p.time_entropy for p in candidates]

        pi_gamma = percentile_ranks(completeness_vals)
        pi_delta = percentile_ranks(diversity_vals)
        pi_sd = percentile_ranks(sd_vals)
        pi_iqr = percentile_ranks(iqr_vals)
        pi_H = percentile_ranks(entropy_vals)

        for p, r_g, r_d, r_sd, r_iqr, r_H in zip(
            candidates, pi_gamma, pi_delta, pi_sd, pi_iqr, pi_H
        ):
            p.s_dt = (
                weights["completeness"] * r_g
                + weights["diversity"] * r_d
                + weights["sd"] * r_sd
                + weights["iqr"] * r_iqr
                + weights["entropy"] * r_H
            )


def mark_candidates(
    df_profiles: pd.DataFrame,
    pk_top_k: int = 3,
    pk_min_distinctness: float = 0.9,
    pk_min_completeness: float = 0.9,
    dt_top_k: int = 3,
    dt_min_parse: float = 0.5,
) -> pd.DataFrame:
    """
    Add boolean columns is_pk_candidate and is_dt_candidate.

    Rules:
    - PK candidates: high distinctness & completeness, top-k by s_pk per table.
    - Datetime candidates: good parse success, top-k by s_dt per table.
    - Exclusivity: a column marked as is_dt_candidate cannot be is_pk_candidate.

    Safe if df_profiles is empty or missing the 'table' column.
    """
    df = df_profiles.copy()

    # If nothing was profiled, just add the candidate columns and return
    if df.empty or "table" not in df.columns:
        df["is_pk_candidate"] = False
        df["is_dt_candidate"] = False
        return df

    df["is_pk_candidate"] = False
    df["is_dt_candidate"] = False

    for table, sub in df.groupby("table"):
        # --- PK candidates: high distinctness & completeness, top-k by s_pk ---
        pk_mask = (
            (sub["distinctness"] >= pk_min_distinctness)
            & (sub["completeness"] >= pk_min_completeness)
        )
        pk_sub = sub.loc[pk_mask].sort_values("s_pk", ascending=False).head(pk_top_k)
        df.loc[pk_sub.index, "is_pk_candidate"] = True

        # --- Datetime candidates: good parse success, top-k by s_dt ---
        dt_mask = (sub["parse_success"] >= dt_min_parse)
        dt_sub = sub.loc[dt_mask].sort_values("s_dt", ascending=False).head(dt_top_k)
        df.loc[dt_sub.index, "is_dt_candidate"] = True

    # --- Enforce exclusivity: datetime candidate cannot be PK candidate ---
    df.loc[df["is_dt_candidate"], "is_pk_candidate"] = False

    return df


def profile_folder(
    folder_path: str,
    weights: Dict[str, float] | None = None,
    parse_threshold: float = 0.5,
    min_distinct_times: int = 2,
) -> pd.DataFrame:
    """
    Profile all .csv files in a folder. Returns a DataFrame with one row per column.
    """
    folder = Path(folder_path)
    all_profiles: List[ColumnProfile] = []

    for csv_path in sorted(folder.glob("*.csv")):
        print(f"Profiling {csv_path.name}")
        try:
            df = pd.read_csv(csv_path)
        except Exception as e:
            print(f"  ! Failed to read {csv_path}: {e}")
            continue

        table_name = csv_path.stem
        all_profiles.extend(profile_table(table_name, df))

    # Compute s_dt for datetime-like columns
    assign_datetime_scores(
        all_profiles,
        weights=weights,
        parse_threshold=parse_threshold,
        min_distinct_times=min_distinct_times,
    )

    records = [asdict(p) for p in all_profiles]
    df_profiles = pd.DataFrame.from_records(records)
    return df_profiles


def profile_folder_with_output(
    folder_path: str,
    output_csv: Optional[str] = None,
    weights: Dict[str, float] | None = None,
    parse_threshold: float = 0.5,
    min_distinct_times: int = 2,
    pk_top_k: int = 3,
    pk_min_distinctness: float = 0.9,
    pk_min_completeness: float = 0.9,
    dt_top_k: int = 3,
    dt_min_parse: float = 0.5,
) -> str:
    """
    Full driver to profile all CSV files and write the summary CSV.
    """
    df_profiles = profile_folder(
        folder_path,
        weights=weights,
        parse_threshold=parse_threshold,
        min_distinct_times=min_distinct_times,
    )

    df_marked = mark_candidates(
        df_profiles,
        pk_top_k=pk_top_k,
        pk_min_distinctness=pk_min_distinctness,
        pk_min_completeness=pk_min_completeness,
        dt_top_k=dt_top_k,
        dt_min_parse=dt_min_parse,
    )

    if output_csv is None:
        output_csv = str(Path(folder_path) / "column_profiling_summary.csv")

    df_marked.to_csv(output_csv, index=False)
    print(f"\nWrote profiling summary to: {output_csv}")
    return output_csv


In [6]:
# Path to the folder containing your CSVs
folder_path = "C:\\Users\\joel\\Desktop\\CAiSE_2\\DGP_1\\data\\food_deliver_clean"   # 👈 change this to your folder
output_csv_path = "C:\\Users\\joel\\Desktop\\CAiSE_2\\DGP_1\\data\\profile_primitives.csv"   # or set to a specific file path string

# folder_path = "C:\\Users\\joel\\Desktop\\CAiSE_2\\TPC_H\\tpch_csv"   # 👈 change this to your folder
# output_csv_path = "C:\\Users\joel\Desktop\CAiSE_2\TPC_H\\output.csv"   # or set to a specific file path string


summary_path = profile_folder_with_output(folder_path, output_csv=output_csv_path)

summary_df = pd.read_csv(summary_path)
summary_df.head()


Profiling Cancellations.csv
Profiling CardUpdates.csv


C:\Users\joel\AppData\Local\Temp\ipykernel_44708\1041876355.py:136: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(series.dropna(), errors="coerce", infer_datetime_format=True)
C:\Users\joel\AppData\Local\Temp\ipykernel_44708\1041876355.py:136: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(series.dropna(), errors="coerce", infer_datetime_format=True)
C:\Users\joel\AppData\Local\Temp\ipykernel_44708\1041876355.py:136: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata

Profiling CardVerifications.csv
Profiling Customers.csv


C:\Users\joel\AppData\Local\Temp\ipykernel_44708\1041876355.py:136: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(series.dropna(), errors="coerce", infer_datetime_format=True)
C:\Users\joel\AppData\Local\Temp\ipykernel_44708\1041876355.py:136: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(series.dropna(), errors="coerce", infer_datetime_format=True)
C:\Users\joel\AppData\Local\Temp\ipykernel_44708\1041876355.py:136: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata

Profiling CustomerValidation.csv
Profiling dataset_summary.csv
Profiling Deliveries.csv


...

C:\Users\joel\AppData\Local\Temp\ipykernel_44708\1041876355.py:136: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(series.dropna(), errors="coerce", infer_datetime_format=True)
C:\Users\joel\AppData\Local\Temp\ipykernel_44708\1041876355.py:136: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(series.dropna(), errors="coerce", infer_datetime_format=True)
C:\Users\joel\AppData\Local\Temp\ipykernel_44708\1041876355.py:136: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata

Profiling Payments.csv


C:\Users\joel\AppData\Local\Temp\ipykernel_44708\1041876355.py:136: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(series.dropna(), errors="coerce", infer_datetime_format=True)
C:\Users\joel\AppData\Local\Temp\ipykernel_44708\1041876355.py:136: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(series.dropna(), errors="coerce", infer_datetime_format=True)
C:\Users\joel\AppData\Local\Temp\ipykernel_44708\1041876355.py:136: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a futur

Profiling ReceiptsSparse.csv
Profiling Regions.csv

Wrote profiling summary to: C:\Users\joel\Desktop\CAiSE_2\DGP_1\data\profile_primitives.csv


,table,column,dtype,n_rows,n_non_null,completeness,distinct_count,distinctness,avg_length,std_length,...,parse_success,n_parsed,distinct_times,time_diversity,time_sd_days,time_iqr_days,time_entropy,s_dt,is_pk_candidate,is_dt_candidate
0,Cancellations,CancelID,object,3661,3661,1.0,3661,1.000000,13.000000,0.000000,...,0.0,0,0,0.0,NaN,NaN,NaN,0.0,True,False
1,Cancellations,OrigOrderKey,object,3661,3661,1.0,3661,1.000000,14.000000,0.000000,...,0.0,0,0,0.0,NaN,NaN,NaN,0.0,True,False
2,Cancellations,CancelledAt,object,3661,3661,1.0,3661,1.000000,32.000000,0.000000,...,1.0,3661,3661,1.0,157.80835,271.665972,8.941319,1.0,False,True
3,Cancellations,Reason,object,3661,3661,1.0,4,0.001093,11.475826,2.228856,...,0.0,0,0,0.0,NaN,NaN,NaN,0.0,False,False
4,CardUpdates,UpdateID,object,3000,3000,1.0,3000,1.000000,13.000000,0.000000,...,0.0,0,0,0.0,NaN,NaN,NaN,0.0,True,False
